In [5]:
import lxml.etree as ET

with open("step1.xml", "rb") as f:
    data = f.read()
    
xml = ET.fromstring(data)
xmlns = "http://www.tei-c.org/ns/1.0"
ns = "{"+xmlns+"}"

for sense in xml.xpath("//t:sense", namespaces={"t":xmlns}):
    group = []
    cit = []
    lg = []
    before = None
    
    for desc in sense.xpath("./node()"):
        if not isinstance(desc, ET._ElementUnicodeResult):
            if desc.tag == ns+"l":
                if before is None:
                    parent = desc.getparent()
                    before = parent.index(desc)
                lg.append(desc)
            elif desc.tag == ns+"bibl":
                # Create both nodes
                cit = ET.Element(ns+"cit")
                quote = ET.Element(ns+"quote")
                lg_node = ET.Element(ns+"lg")
                
                # Add <lg> to <cit>
                cit.append(quote)
                quote.append(lg_node)
                # Add each line
                for l in lg:
                    lg_node.append(l)
                # Add the bibliography
                cit.append(desc)
                
                # Insert in the tree
                parent.insert(before, cit)
                
                cit.tail = desc.tail
                desc.tail = ""
                
                # Reset
                lg = []
                before = None


string = ET.tostring(xml, method='xml', pretty_print=True, encoding=str)
string =string.replace("(", "<cit><bibl>").replace(")", "</bibl></cit>")

with open("step2.xml", "w") as f:
    f.write(string)
                